In [9]:
import pickle

import pandas as pd
import scipy.sparse
import scipy.sparse

from lightfm import LightFM
from lightfm.data import Dataset

In [10]:
df = pd.read_parquet('/pio/scratch/1/recommender_systems/processed/amazon-clothes/5-core/train.parquet')
test_df = pd.read_parquet('/pio/scratch/1/recommender_systems/processed/amazon-clothes/5-core/test.parquet')
categories = pd.read_json('/pio/scratch/1/recommender_systems/interim/Amazon/meta_Clothing_Shoes_and_Jewelry_categories.json', lines=True)

In [11]:
interactions = scipy.sparse.load_npz('/pio/scratch/1/i313924/data/lightfm_data/5_core_interactions.npz')
test_interactions = scipy.sparse.load_npz('/pio/scratch/1/i313924/data/lightfm_data/5_core_test_interactions.npz')
dataset = pd.read_pickle('/pio/scratch/1/i313924/data/lightfm_data/5_core_dataset_features.pkl')
item_features = pd.read_pickle('/pio/scratch/1/i313924/data/lightfm_data/5_core_features.pkl')

In [12]:
word2vec_embeddings = pd.read_parquet('/pio/scratch/1/i313924/data/word2vec/word2vec_240_embeddings_categories.parquet')

In [13]:
word2vec_embeddings

,0,1,2,3,4,5,6,7,8,9,...,230,231,232,233,234,235,236,237,238,239
Women,-0.008189,0.022066,-0.026420,0.094180,-0.062463,-0.018453,-0.086254,0.094877,-0.074483,-0.057344,...,0.049633,0.079719,0.061025,0.039601,0.035771,0.017603,-0.005381,-0.062055,-0.041591,-0.055139
Men,-0.137466,-0.094954,0.053687,0.040817,-0.008410,-0.004553,0.033081,-0.062215,-0.023763,-0.034139,...,-0.101567,-0.049388,-0.049279,-0.058980,0.077279,-0.012332,-0.005987,-0.026190,-0.077885,-0.017687
B00SU52460,0.080074,-0.029241,-0.012169,0.001674,-0.127657,0.031498,0.066277,-0.010632,-0.067872,-0.003321,...,-0.011278,-0.006868,0.028322,0.013549,0.053158,0.032340,0.044718,-0.016185,0.061387,-0.001913
B00STBRT9M,0.006809,-0.095584,-0.066293,-0.048665,0.012165,-0.019132,-0.095805,0.072027,-0.033602,0.014023,...,0.022977,0.060151,-0.096941,0.111090,-0.058104,-0.024142,-0.084117,0.068409,0.113990,-0.040187
B00SU5244M,-0.163030,-0.030770,-0.039648,0.183576,-0.081303,-0.031919,0.066622,0.044468,0.017939,0.003620,...,0.044207,-0.026794,-0.051534,0.063143,-0.124244,-0.021593,-0.020266,0.043678,-0.061697,-0.026006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B01D2H01KG,0.050055,-0.036643,-0.148640,-0.049264,0.035507,-0.154216,0.063264,-0.063919,0.007130,0.086678,...,0.069495,0.052381,-0.075635,0.052993,-0.032045,-0.011593,-0.033095,0.025782,-0.087101,-0.031072
B005AR7V1Y,-0.064120,-0.114724,0.044103,-0.058813,-0.012883,-0.075538,-0.035813,-0.038642,-0.020050,0.013223,...,-0.004069,0.034750,0.012952,0.113518,-0.022946,-0.095672,-0.045898,0.020638,-0.003015,-0.032189
B00K59XJ40,-0.007995,0.010419,0.014158,0.027516,-0.011702,0.003093,-0.003959,-0.085708,-0.091460,0.061226,...,-0.038815,0.011818,0.032563,0.036898,0.072990,-0.032794,0.002982,-0.143381,0.034707,-0.108142
B00HEFIHJU,-0.075908,-0.026579,-0.027001,0.014267,-0.075832,-0.010865,-0.013595,-0.055790,-0.012770,-0.037490,...,0.023234,0.003851,0.076986,0.000565,-0.069062,-0.004645,0.098052,0.033378,0.102681,-0.050824


In [14]:
item_mapping = dataset.mapping()[3]
word2vec_embeddings = word2vec_embeddings.rename(index=item_mapping)
word2vec_embeddings = word2vec_embeddings.sort_index()

In [15]:
model = LightFM(
    no_components=240,
    loss='warp',
    learning_schedule='adadelta',
    epsilon=2.45e-07,
    rho=0.958,
    # item_alpha=5.97e-05,
    user_alpha=2.06e-06,
    max_sampled=9
)

my new implementation


In [16]:
num_epochs = [5, 25, 50, 250, 500, 1000]
remaining = [num_epochs[0]] + [num_epochs[i] - num_epochs[i-1] for i in range(1, len(num_epochs))]
for i, epochs in enumerate(num_epochs):
    model.fit_partial(interactions, item_features=item_features, verbose=True, epochs=remaining[i], num_threads=8, word2vec_embeddings=word2vec_embeddings.values)
    pickle.dump(model, open(f'/pio/scratch/1/i313924/data/lightfm_data/model_{epochs}_word2vec_categories_item_alpha_0.pkl', 'wb'), protocol=4)

Epoch: 100%|██████████| 500/500 [1:11:59<00:00,  8.64s/it]


In [18]:
2

2

2